<a href="https://colab.research.google.com/github/Jenish-lezdo/ml-learning/blob/main/Similarity_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers fitz
!pip install pymupdf
!pip install faiss-cpu torchvision torch numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35615 sha256=4bf8dc5a2b07f6d6097040c349f0f5abd6e50ae2263076d184bbeca

In [ ]:
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF
import faiss

# Load the model
# model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model = SentenceTransformer("intfloat/multilingual-e5-large")
# model = SentenceTransformer("abhinand/MedEmbed-large-v0.1")
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


# Step 1: Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text_content = []
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page_text = pdf[page_num].get_text()
            text_content.append(page_text)
    return text_content

# Step 2: Encode extracted text
pdf_path = "./files/patient3.pdf"  # Replace with your PDF file path
sentences = ["The doctor made a quick diagnosis after reviewing the patient's symptoms.", "The COVID-19 vaccine has been widely distributed to reduce the spread of the virus.", "Immunization against the flu is recommended every year.","The prescribed treatment involves a combination of medication and therapy."]
# sentences = extract_text_from_pdf(pdf_path)
embeddings = model.encode(sentences).astype('float32')
# Initialize a FAISS index for exact search (IndexFlatL2)
# Define the dimension

dimension = model.get_sentence_embedding_dimension()
# dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# Step 3: Define a query and calculate similarities
query = "DOCTOR"
query_embedding = model.encode([query]).astype('float32')
k = 1  # Number of nearest neighbors
# Step 4: Calculate cosine similarity between query and document embeddings
# similarities = util.cos_sim(query_embedding, embeddings)
similarities = index.search(query_embedding, k)
# Print similarities for each document segment with the query
print(similarities)
distances, indices = index.search(query_embedding, k)
print("Most similar sentence:", sentences[indices[0][0]])


(array([[0.33697003]], dtype=float32), array([[0]]))
Most similar sentence: The doctor made a quick diagnosis after reviewing the patient's symptoms.


In [ ]:
! pip install indic-transliteration  ##for fine tuning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.7 MB/s eta 0:00:00


Fine TUne a Model

In [ ]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

# Assume you have a list of InputExample objects with Tanglish-Tamil pairs
train_examples = [
    InputExample(texts=["intru vanilai arumaiyaka ullathu", "இன்று வானிலை அருமையாக உள்ளது"]),
    # Add more examples...
]

# Load a pre-trained multilingual model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Create a dataset and dataloader
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# Define a loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

# Save the fine-tuned model
model.save('fine-tuned-model')


NameError: name 'Dataset' is not defined

In [ ]:
for i, text in enumerate(sentences):
    print(f"Page {i + 1} Text:")
    print(text)
    print("\nEmbeddings:\n", embeddings[i], "\n")

Page 1 Text:
Imaginary Hospital - Patient Record
Patient Name: John Doe
Age: 45
Gender: Male
Patient ID: 123456789
Contact: (123) 456-7890
Medical History:
- Hypertension diagnosed in 2010
- Type 2 Diabetes diagnosed in 2015
- Surgery: Appendectomy in 2012
- Allergy to penicillin
Current Condition:
Diagnosis: Chronic Kidney Disease
Symptoms: Fatigue, Swelling in lower limbs, Shortness of breath
Severity: Moderate
Treatment Plan:
- Medication: ACE Inhibitors
- Low-sodium diet
- Weekly dialysis sessions
- Physical therapy for improved mobility
Doctor's Notes:
Page 1


Embeddings:
 [-2.51276884e-02  4.28194329e-02  6.59860820e-02  4.28432412e-02
 -1.67175699e-02  1.46574266e-02  6.82771876e-02  5.48424348e-02
 -4.11398932e-02  8.12840089e-03  1.36558274e-02  9.51576605e-03
  2.06603575e-03  1.42450705e-02 -3.38468514e-02  3.55952457e-02
 -1.48814227e-02 -4.50901091e-02 -8.77748802e-02  8.20942149e-02
  7.84432888e-03 -7.71832839e-03  2.14566216e-02 -2.31162310e-02
 -1.22175110e-03 -2.6724

SIMILARITY SEARCH USING FIAS

In [ ]:
!pip install faiss-cpu torchvision torch numpy


  Using cached faiss_cpu-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.3 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import faiss

# Load a pre-trained model (ResNet) and remove the final classification layer
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # Remove the last layer
model.eval()

# Transformation for images (resize and normalize to ResNet input requirements)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to generate embeddings from an image
def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = model(image).squeeze().numpy()  # Convert to 1D array
    return embedding.astype('float32')  # FAISS needs float32
